<a href="https://colab.research.google.com/github/bjyberg/UAV_PeatlandMonitoring/blob/main/DL_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 20.9 MB 84.8 MB/s 


In [ ]:
!pip install ipympl

In [61]:
%matplotlib widget

In [3]:
import tensorflow as tf
from tensorflow import keras
import datetime
import numpy as np
import rasterio
import sklearn
import os
import glob
from matplotlib import pyplot as plt

In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
sizeX = 64
sizeY = 64
N_class = 7

In [6]:
def load_tiled_data(folder_name):
  path = os.listdir(folder_name)
  tiles = []
  for file in glob.glob(folder_name + '/*.tif'):
    image = rasterio.open(file)
    patch = image.read()
    tiles.append(patch)
  return(np.array(tiles))

In [7]:
patch_array = load_tiled_data('/content/drive/MyDrive/CNN/training/patched_images')

In [8]:
lab_array = load_tiled_data('/content/drive/MyDrive/CNN/training/patched_labels')

In [9]:
np.unique(lab_array)

array([0, 1, 2, 3, 4, 5, 6], dtype=uint8)

In [10]:
print(patch_array.shape)
print(lab_array.shape)

(110, 5, 64, 64)
(110, 1, 64, 64)


In [16]:
patch_array = patch_array.reshape(110,64,64,5)
lab_array = lab_array.reshape(110,64,64,1)

In [17]:
patch_array = keras.utils.normalize(patch_array)

In [20]:
one_hot_train_labs = keras.utils.to_categorical(lab_array, num_classes=N_class)

In [21]:
one_hot_train_labs.shape

(110, 64, 64, 7)

In [ ]:
#We need to flatten everything in one hot train labs if we do this... 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                  np.unique(one_hot_train_labs),
                                                  one_hot_train_labs)

In [30]:
val_patch_array = load_tiled_data('/content/drive/MyDrive/CNN/validation/patched_images')
val_lab_array = load_tiled_data('/content/drive/MyDrive/CNN/validation/patched_labels')


In [32]:
print(val_patch_array.shape)
print(val_lab_array.shape)

(28, 5, 64, 64)
(28, 1, 64, 64)


In [33]:
val_patch_array = val_patch_array.reshape(28,64,64,5)
val_lab_array = val_lab_array.reshape(28,64,64,1)
one_hot_val_labs = keras.utils.to_categorical(val_lab_array, num_classes=N_class)

In [42]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, Dropout

def multi_unet_model(n_classes, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    #Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
     
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    
    #NOTE: Compile the model in the main program to make it easy to test with various loss functions
    #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    #model.summary()
    
    return model

In [43]:
model = multi_unet_model(7, 64, 64, 5)

In [44]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 64, 64, 5)]  0           []                               
                                                                                                  
 conv2d_19 (Conv2D)             (None, 64, 64, 16)   736         ['input_2[0][0]']                
                                                                                                  
 dropout_9 (Dropout)            (None, 64, 64, 16)   0           ['conv2d_19[0][0]']              
                                                                                                  
 conv2d_20 (Conv2D)             (None, 64, 64, 16)   2320        ['dropout_9[0][0]']              
                                                                                            

In [47]:
history = model.fit(patch_array, one_hot_train_labs, 
                    batch_size = 8, 
                    verbose=1, 
                    epochs=500, 
                    validation_data=(val_patch_array, one_hot_val_labs), 
                    #class_weight=class_weights,
                    shuffle=False)

Epoch 1/500
14/14 [==============================] - 2s 82ms/step - loss: 0.8298 - accuracy: 0.7796 - val_loss: 3495.0935 - val_accuracy: 0.7867
Epoch 2/500
14/14 [==============================] - 0s 14ms/step - loss: 0.8367 - accuracy: 0.7796 - val_loss: 3196.6770 - val_accuracy: 0.7867
Epoch 3/500
14/14 [==============================] - 0s 13ms/step - loss: 0.8290 - accuracy: 0.7796 - val_loss: 3392.0769 - val_accuracy: 0.7867
Epoch 4/500
14/14 [==============================] - 0s 13ms/step - loss: 0.8277 - accuracy: 0.7796 - val_loss: 3388.0212 - val_accuracy: 0.7867
Epoch 5/500
14/14 [==============================] - 0s 13ms/step - loss: 0.8282 - accuracy: 0.7796 - val_loss: 3373.0923 - val_accuracy: 0.7867
Epoch 6/500
14/14 [==============================] - 0s 13ms/step - loss: 0.8272 - accuracy: 0.7796 - val_loss: 3393.8120 - val_accuracy: 0.7867
Epoch 7/500
14/14 [==============================] - 0s 14ms/step - loss: 0.8262 - accuracy: 0.7796 - val_loss: 3391.3108 - val_ac